In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
%matplotlib inline

In [2]:
date_columns = ['CREATED','UPDATED', 'DELETED']
df = pd.read_csv("postclick_test.csv", sep = '[/|]', engine = 'python', parse_dates=date_columns)
df.head(5)

,FILE_ID,Unnamed: 1,Unnamed: 2,USER,Unnamed: 4,Unnamed: 5,CREATED,Unnamed: 7,Unnamed: 8,UPDATED,Unnamed: 10,Unnamed: 11,DELETED,Unnamed: 13,Unnamed: 14,VERSION
0,rrT0615,NaN,NaN,3633870,NaN,NaN,2019-08-01,NaN,NaN,2019-08-14,NaN,NaN,NaT,NaN,NaN,26
1,rrT0625,NaN,NaN,2853751,NaN,NaN,2019-08-01,NaN,NaN,2019-11-19,NaN,NaN,1970-01-01,NaN,NaN,26
2,rrT0630,NaN,NaN,2538826,NaN,NaN,2019-08-01,NaN,NaN,2019-09-28,NaN,NaN,NaT,NaN,NaN,2
3,rrT0635,NaN,NaN,1642941,NaN,NaN,2019-08-01,NaN,NaN,2019-12-05,NaN,NaN,NaT,NaN,NaN,5
4,rrT0640,NaN,NaN,3635605,NaN,NaN,2019-08-01,NaN,NaN,2019-10-13,NaN,NaN,2019-10-13,NaN,NaN,1


In [3]:
df.drop(df.columns[[1,2,4,5,7,8,10,11,13,14]], axis=1, inplace=True)
df.head(5)

,FILE_ID,USER,CREATED,UPDATED,DELETED,VERSION
0,rrT0615,3633870,2019-08-01,2019-08-14,NaT,26
1,rrT0625,2853751,2019-08-01,2019-11-19,1970-01-01,26
2,rrT0630,2538826,2019-08-01,2019-09-28,NaT,2
3,rrT0635,1642941,2019-08-01,2019-12-05,NaT,5
4,rrT0640,3635605,2019-08-01,2019-10-13,2019-10-13,1


In [4]:
# lets rename the headers
df = df.rename(columns = {'FILE_ID':'file_id','USER':'user_id','CREATED':'created','UPDATED':'updated'
                          ,'DELETED':'deleted','VERSION':'version'})
df.head(1)

,file_id,user_id,created,updated,deleted,version
0,rrT0615,3633870,2019-08-01,2019-08-14,NaT,26


In [5]:
df.shape

(102133, 6)

In [6]:
# Check to see if file_id is a unique value
df['file_id'].nunique()

# There are 102133 unique file id which matches the count above so file_id has unique values.

102133

In [7]:
df.isnull().sum()

file_id        0
user_id        4
created        0
updated        0
deleted    73795
version       30
dtype: int64

## Data Cleaning: <span style="color:blue">Null Values</span>

#### We can remove the 4 missing values for user since 4 is not a large number of missing values. Version is the number of times a file has been updated. We can either fill the null values with the mean or median, but since there are only 30 missing values out of over 100k+ data I will remove them.

In [8]:
# In this scenario we do not to delete rows where deleted is null. We can leave it alone.
df = df.dropna(subset=['user_id', 'version'])
df.isnull().sum()

file_id        0
user_id        0
created        0
updated        0
deleted    73775
version        0
dtype: int64

## Data Cleaning: <span style="color:blue">Update dates</span>

#### Deleted has a date as early as 1970 when the dataset is supposed to have files created between aug 1st 2019 and Oct 31st 2019

In [9]:
df.describe()

,file_id,user_id,created,updated,deleted,version
count,102099,102099,102099,102099,28324,102099
unique,102099,10035,92,188,189,420
top,17974415,3282991,2019-08-13 00:00:00,2019-08-13 00:00:00,2019-10-25 00:00:00,1
freq,1,6038,5123,4405,474,27472
first,NaN,NaN,2019-08-01 00:00:00,2019-08-01 00:00:00,1970-01-01 00:00:00,NaN
last,NaN,NaN,2019-10-31 00:00:00,2020-02-04 00:00:00,2020-02-04 00:00:00,NaN


In [10]:
# Lets see how many rows has a date earlier than 8/1/2019
print(df[df['deleted']< '2019-08-01'])

         file_id  user_id    created    updated    deleted version
1        rrT0625  2853751 2019-08-01 2019-11-19 1970-01-01      26
500      KLp3685  2728736 2019-08-01 2019-08-01 1970-01-01       2
569      KLp4075  3161776 2019-08-01 2019-12-09 1970-01-01       2
597      KLp4290   262451 2019-08-01 2019-08-21 1970-01-01       2
2482    ffR12445    43539 2019-08-02 2019-10-14 1970-01-01       3
...          ...      ...        ...        ...        ...     ...
100308  Gty08340  2784921 2019-10-30 2019-11-25 1970-01-01      18
100448  Gty09150  2750096 2019-10-30 2019-12-18 1970-01-01       4
101091  Gty26545  3641515 2019-10-31 2019-10-31 1970-01-01       2
101302  Gty27735  2750096 2019-10-31 2019-11-15 1970-01-01     126
101860  Gty30960  3412780 2019-10-31 2019-12-18 1970-01-01      59

[259 rows x 6 columns]


In [11]:
# Check to see if there are 259 rows with 1970 dates
print(len(df[df['deleted']== '1970-01-01']))

259


#### There are 259 rows that has deleted date in 1970. This is clearly wrong. If we take a look at a few rows, it looks like deleted date is the same as updated date.

In [12]:
df[df['deleted'].notnull()].head(20)

,file_id,user_id,created,updated,deleted,version
1,rrT0625,2853751,2019-08-01,2019-11-19,1970-01-01,26
4,rrT0640,3635605,2019-08-01,2019-10-13,2019-10-13,1
5,rrT0645,268126,2019-08-01,2019-11-08,2019-11-08,49
11,rrT0860,3012311,2019-08-01,2019-08-01,2019-08-01,1
18,rrT1460,3628745,2019-08-01,2019-08-01,2019-08-01,1
32,rrT2990,3301506,2019-08-01,2019-08-01,2019-08-01,1
33,rrT3115,3628745,2019-08-01,2019-08-01,2019-08-01,3
43,rrT3575,3635605,2019-08-01,2019-10-13,2019-10-13,1
44,rrT3605,3635690,2019-08-01,2019-10-14,2019-10-14,1
45,rrT3620,317040,2019-08-01,2019-08-01,2019-08-01,1


In [13]:
# Lets confirm how many rows where deleted date equals updated date.
# This will confirm whether we should update deleted date to match updated date.

print(len(df[df['deleted']==df['updated']]))

# With 28012 match this is a good indication to change the 1970 dates to match the updated dates.

28012


#### The summary indicates that we should have 28324 deleted dates. 28012 + 259 is only 28271. Looks like there are 53 rows where deleted date does not equal updated date. Lets focus on the 1970 dates first.

In [14]:
# See which file id has 1970 deleted date. That way when we make the changes we can double check if the changes 
# were done correctly

file_1970 = df[df['deleted']=='1970-01-01']
file_1970.head(10)

,file_id,user_id,created,updated,deleted,version
1,rrT0625,2853751,2019-08-01,2019-11-19,1970-01-01,26
500,KLp3685,2728736,2019-08-01,2019-08-01,1970-01-01,2
569,KLp4075,3161776,2019-08-01,2019-12-09,1970-01-01,2
597,KLp4290,262451,2019-08-01,2019-08-21,1970-01-01,2
2482,ffR12445,43539,2019-08-02,2019-10-14,1970-01-01,3
3468,ffR32025,872068,2019-08-05,2019-09-19,1970-01-01,13
4245,ffR36495,2629501,2019-08-05,2019-12-12,1970-01-01,17
4247,ffR36505,2871311,2019-08-05,2019-12-04,1970-01-01,5
4277,ffR36680,2871311,2019-08-05,2019-12-04,1970-01-01,3
5008,ffR48990,180880,2019-08-06,2019-08-06,1970-01-01,2


#### Create a list to store all file_id with 1970 deleted date. We will use the list to check if the dates updated like how we want them to.

In [15]:
list_1970 = file_1970["file_id"].tolist()
print(list_1970)

['rrT0625', 'KLp3685', 'KLp4075', 'KLp4290', 'ffR12445', 'ffR32025', 'ffR36495', 'ffR36505', 'ffR36680', 'ffR48990', 'ffR49610', 'ffR52760', 'ffR53045', 'ffR60175', 'ffR62380', 'ffR63930', 'ffR65135', 'ffR65675', 'ffR67195', 'ffR78860', 'ffR78960', 'ffR80080', 'ffR81635', 'ffR83fgT', 'ffR97325', 'ffR99505', '17500390', '175ddR30', '17532010', '17532015', '17532025', '17532030', '17532050', '17537505', '17549290', '17549440', '17592375', '175933pp', '17599400', '17601065', '17604025', '17625030', '17627085', '17627515', '17628080', '17647860', '17648520', '17725280', '17727940', '17733800', '17746735', '17749840', '17749945', '17750075', '17750530', '17750715', '17752495', '17752865', '17768970', '17769215', '17769880', '17770235', '17771445', '17774360', '17775010', '17791835', '17793325', '17795030', '17795040', '17795445', '17798015', '17798035', '178139pp', '17813995', '17818250', '17827540', '17827pp0', '17831960', '17849670', '17850035', '17850045', '17850975', '17852720', '17853f

#### Update 1970 deleted dates to match updated dates

In [16]:
df.loc[df['deleted']=="1970-01-01", 'deleted'] = df['updated']

In [17]:
# Lets check to see if we made the changes correctly. Refer back to list_1970. 
# That list has the file id of all 1970 deleted dates before we made the changes.

check_df = df[df['file_id'].isin(list_1970)]
check_df.head(10)

# Looking at the top 10 rows we can confirm that it was done correctly.

,file_id,user_id,created,updated,deleted,version
1,rrT0625,2853751,2019-08-01,2019-11-19,2019-11-19,26
500,KLp3685,2728736,2019-08-01,2019-08-01,2019-08-01,2
569,KLp4075,3161776,2019-08-01,2019-12-09,2019-12-09,2
597,KLp4290,262451,2019-08-01,2019-08-21,2019-08-21,2
2482,ffR12445,43539,2019-08-02,2019-10-14,2019-10-14,3
3468,ffR32025,872068,2019-08-05,2019-09-19,2019-09-19,13
4245,ffR36495,2629501,2019-08-05,2019-12-12,2019-12-12,17
4247,ffR36505,2871311,2019-08-05,2019-12-04,2019-12-04,5
4277,ffR36680,2871311,2019-08-05,2019-12-04,2019-12-04,3
5008,ffR48990,180880,2019-08-06,2019-08-06,2019-08-06,2


#### Lets circle back to the 53 rows where updated date does not equal deleted date.

In [18]:
not_null_deleted = df[df['deleted'].notnull()]
not_null_deleted = not_null_deleted[not_null_deleted['deleted']!=not_null_deleted['updated']]
print(not_null_deleted)

         file_id  user_id    created    updated    deleted version
1476    ffR06450  3179881 2019-08-02 2019-11-12 2019-08-06       1
2639    ffR178pp  3601670 2019-08-03 2019-08-05 2019-08-04       1
6138    ffR60680  3245996 2019-08-07 2019-12-20 2019-09-21      36
9275    ffR97900   909363 2019-08-09 2019-10-22 2019-10-21       3
10225   175178pp  1257103 2019-08-10 2019-10-14 2019-09-30      28
10967   17549340  2702756 2019-08-12 2019-12-10 2019-10-15       2
12004   17pp5065  3640pp5 2019-08-12 2019-10-04 2019-08-12       1
14683   17575615  3626420 2019-08-13 2019-09-13 2019-09-11       3
20081   17626535  3139506 2019-08-14 2019-08-19 2019-08-15       1
20217   17627295  3325386 2019-08-14 2019-12-20 2019-12-19       4
21716   17650345  3637630 2019-08-15 2019-10-08 2019-08-29      12
21738   17650515  3637630 2019-08-15 2019-10-08 2019-08-29       1
26465   17751340  1431233 2019-08-20 2019-09-11 2019-08-26      39
26843   17767510  1044328 2019-08-21 2019-12-04 2019-11-13    

#### The above output shows deleted date being less than updated date. This does not make sense logically and since we confirmed that updated dates should equal deleted dates, that is what we will do here as well.

In [19]:
list_53 = not_null_deleted['file_id'].tolist()
print(list_53)

['ffR06450', 'ffR178pp', 'ffR60680', 'ffR97900', '175178pp', '17549340', '17pp5065', '17575615', '17626535', '17627295', '17650345', '17650515', '17751340', '17767510', '17769270', '17769685', '17769720', '17772915', '17773500', '17847665', '17847715', '17847745', '17847750', '17863350', '17864905', '17867940', '17899020', '1797Gty0', '18092100', '18130675', '18130745', '18132415', '18230370', '18230585', '18382730', '1840pp50', '18429000', '18496275', '18497635', 'fgT01710', 'fgT33510', 'fgT52775', '18685145', 'ddR37750', 'ddR41195', 'ddR41250', 'ddR41265', 'ddR75985', 'ddR76030', 'nnJ66925', 'nnJ87335', 'nnJ88270', 'Gty09630']


In [20]:
df['deleted'] = df.apply(lambda x: x['updated'] if (np.all(pd.notnull(x['deleted']))) else x['deleted'], axis=1)

# This code would have resolved our 259 count of 1970 dates as well had we checked why there were 53 rows where 
# updated dates did not equal deleted dates. Would have been less work.

In [21]:
# check to see if the changes were done correctly
df[df['file_id'].isin(list_53)].head(10)

,file_id,user_id,created,updated,deleted,version
1476,ffR06450,3179881,2019-08-02,2019-11-12,2019-11-12,1
2639,ffR178pp,3601670,2019-08-03,2019-08-05,2019-08-05,1
6138,ffR60680,3245996,2019-08-07,2019-12-20,2019-12-20,36
9275,ffR97900,909363,2019-08-09,2019-10-22,2019-10-22,3
10225,175178pp,1257103,2019-08-10,2019-10-14,2019-10-14,28
10967,17549340,2702756,2019-08-12,2019-12-10,2019-12-10,2
12004,17pp5065,3640pp5,2019-08-12,2019-10-04,2019-10-04,1
14683,17575615,3626420,2019-08-13,2019-09-13,2019-09-13,3
20081,17626535,3139506,2019-08-14,2019-08-19,2019-08-19,1
20217,17627295,3325386,2019-08-14,2019-12-20,2019-12-20,4


In [22]:
# This will confirm that all our deleted dates are now correct. 
# We should have 28324 rows where the deleted dates matches the updated dates.

print(len(df[df['deleted']==df['updated']]))

28324


## Data Cleaning: <span style="color:blue">Update Datatype</span>

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102099 entries, 0 to 102132
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   file_id  102099 non-null  object        
 1   user_id  102099 non-null  object        
 2   created  102099 non-null  datetime64[ns]
 3   updated  102099 non-null  datetime64[ns]
 4   deleted  28324 non-null   datetime64[ns]
 5   version  102099 non-null  object        
dtypes: datetime64[ns](3), object(3)
memory usage: 10.5+ MB


#### Lets convert version to integer. Version is the number of times the file was updated and therefore we want to convert to integer

In [24]:
df['version'] = df['version'].astype(int)
# got an error: invalid literal for int() with base 10: 'pp'. Looks like not all versions are numbers

ValueError: invalid literal for int() with base 10: 'pp'

In [25]:
#lets look at unique values for version
df['version'].unique()

array(['26', '2', '5', '1', '49', '8', '3', '27', '14', '11', '137', '9',
       '20', '30', '23', '56', '61', '4', '6', '24', '7', '76', '13',
       '45', '29', '16', '35', '12', '85', '21', '46', '10', '43', '79',
       '80', '123', '51', '17', '50', '292', '404', '192', '63', '28',
       '44', 'pp', '19', '38', '156', '18', '120', '113', '92', '15',
       '86', '52', '126', '25', '47', '70', '71', '111', '178', '53',
       '60', '31', '64', '95', '87', '22', '237', '72', '210', '213',
       '34', '40', '121', '99', '179', '37', '75', '67', '36', '57', '33',
       '69', '269', '119', '66', '78', '106', '32', '41', '48', '62',
       '211', '94', '59', '96', '39', '193', '42', '217', '73', '54',
       '89', '104', '125', '107', '154', '173', '88', '227', '102', '132',
       '65', '196', '169', '83', '136', '287', '281', '116', '253', '133',
       '420', '58', '124', '101', '117', '151', '145', '141', '261',
       '207', '352', '112', '316', '146', '81', '74', '160', '148', 

In [26]:
# At a glance the output above shows there are version with letters
# Check how many rows where there are letters in version
print(df[df['version'].str.contains('[a-zA-Z]',na=False)])

         file_id  user_id    created    updated    deleted version
142      KLp1645  3372380 2019-08-01 2019-11-16        NaT      pp
469      KLp3510   300335 2019-08-01 2019-08-06        NaT      pp
1897    ffR09060  3428805 2019-08-02 2019-08-03 2019-08-03      pp
2851    ffR19605   197190 2019-08-04 2019-10-14        NaT      pp
4092    ffR35610  2321212 2019-08-05 2019-11-05        NaT     fgT
...          ...      ...        ...        ...        ...     ...
100358  Gty08640  3512456 2019-10-30 2019-11-22        NaT     fgT
100457  Gty09195  3672465 2019-10-30 2019-12-27        NaT     fgT
100608  Gty10050  2648936 2019-10-30 2019-10-31        NaT      pp
101177  Gty27050  3497308 2019-10-31 2019-12-10        NaT     Gty
101333  Gty27905  2117066 2019-10-31 2019-12-05        NaT      pp

[191 rows x 6 columns]


#### There are 191 rows where we do not have an exact value for how many times a file has been updated because there are letters were using in place of numbers. Lets remove them since that won't be useful in our analysis and so we can update the data type to integer.

In [27]:
# create list of unique values where version has letters in them
bad_version = df[df['version'].str.contains('[a-zA-Z]',na=False)]['version'].unique().tolist()
print(bad_version)

['pp', 'fgT', '1pp', '4pp', 'ffR', 'nnJ', '3pp', 'Gty', 'ddR', '2pp']


In [28]:
# remove rows with versions listed in bad_version
df = df[~df['version'].isin(bad_version)]

In [29]:
df.isnull().sum()

file_id        0
user_id        0
created        0
updated        0
deleted    73604
version        0
dtype: int64

In [30]:
# Change version data type to integer
df['version'] = df['version'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101908 entries, 0 to 102132
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   file_id  101908 non-null  object        
 1   user_id  101908 non-null  object        
 2   created  101908 non-null  datetime64[ns]
 3   updated  101908 non-null  datetime64[ns]
 4   deleted  28304 non-null   datetime64[ns]
 5   version  101908 non-null  int32         
dtypes: datetime64[ns](3), int32(1), object(2)
memory usage: 5.1+ MB


In [31]:
df.reset_index(inplace=True, drop=True)

## Data Cleaning: <span style="color:blue">Summary</span>
1. Removed null values in user and version since the number of values missing were low.

2. Updated deleted column to reflect the correct dates.

3. Update data type for version. Ran into an issue where some version had mixed numbers and letters. Those were removed to allow us to change data type from object to integer.

In [32]:
df.to_csv("cleaned_take_home_test.csv")